In [2]:
from typing import TypedDict,Annotated,Optional
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,END
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
import json 

In [3]:
load_dotenv()

True

In [4]:
llm = ChatGroq(model="llama-3.1-8b-instant")
llm.invoke("hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.013552101, 'prompt_time': 0.00224824, 'queue_time': 0.041862839, 'total_time': 0.015800341}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run--300b4a89-e83f-4e87-8500-30994f5b98b4-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})

In [ ]:
search_tool = TavilySearchResults(max_results=1)
tools = [search_tool]
memory = MemorySaver()
llm_with_tools = llm.bind_tools(tools=tools)
